In [1]:
import numpy as np
import csv
import re
def set_of_words2vec(vocab_list, input_set):
    """
    遍历查看该单词是否出现，出现该单词则将该单词置1
    :param vocab_list: 所有单词集合列表
    :param input_set: 输入数据集
    :return: 匹配列表[0,1,0,1...]，其中 1与0 表示词汇表中的单词是否出现在输入的数据集中
    """
    # 创建一个和词汇表等长的向量，并将其元素都设置为0
    result = [0] * len(vocab_list)
    # 遍历文档中的所有单词，如果出现了词汇表中的单词，则将输出的文档向量中的对应值设为1
    for word in input_set:
        if word in vocab_list:
            result[vocab_list.index(word)] = 1
        else:
            # 这个后面应该注释掉，因为对你没什么用，这只是为了辅助调试的
            # print('the word: {} is not in my vocabulary'.format(word))
            pass
    return result


def _train_naive_bayes(train_mat, train_category):
    """
    朴素贝叶斯分类原版
    :param train_mat:  type is ndarray
                    总的输入文本，大致是 [[0,1,0,1], [], []]
    :param train_category: 文件对应的类别分类， [0, 1, 0],
                            列表的长度应该等于上面那个输入文本的长度
    :return:
    """
    train_doc_num = len(train_mat)
    words_num = len(train_mat[0])
    # 因为侮辱性的被标记为了1， 所以只要把他们相加就可以得到侮辱性的有多少
    # 侮辱性文件的出现概率，即train_category中所有的1的个数，
    # 代表的就是多少个侮辱性文件，与文件的总数相除就得到了侮辱性文件的出现概率
    pos_abusive = np.sum(train_category) / train_doc_num
    # 单词出现的次数
    # 原版
    p0num = np.zeros(words_num)
    p1num = np.zeros(words_num)

    # 整个数据集单词出现的次数（原来是0，后面改成2了）
    p0num_all = 0
    p1num_all = 0

    for i in range(train_doc_num):
        # 遍历所有的文件，如果是侮辱性文件，就计算此侮辱性文件中出现的侮辱性单词的个数
        if train_category[i] == 1:
            p1num += train_mat[i]
            p1num_all += np.sum(train_mat[i])
        else:
            p0num += train_mat[i]
            p0num_all += np.sum(train_mat[i])
    # 后面需要改成改成取 log 函数
    p1vec = p1num / p1num_all
    p0vec = p0num / p0num_all
    return p0vec, p1vec, pos_abusive


def train_naive_bayes(train_mat, train_category):
    """
    朴素贝叶斯分类修正版，　注意和原来的对比，为什么这么做可以查看书
    :param train_mat:  type is ndarray
                    总的输入文本，大致是 [[0,1,0,1], [], []]
    :param train_category: 文件对应的类别分类， [0, 1, 0],
                            列表的长度应该等于上面那个输入文本的长度
    :return:
    """
    train_doc_num = len(train_mat)
    words_num = len(train_mat[0])
    # 因为侮辱性的被标记为了1， 所以只要把他们相加就可以得到侮辱性的有多少
    # 侮辱性文件的出现概率，即train_category中所有的1的个数，
    # 代表的就是多少个侮辱性文件，与文件的总数相除就得到了侮辱性文件的出现概率
    pos_abusive = np.sum(train_category) / train_doc_num
    # 单词出现的次数
    # 原版，变成ones是修改版，这是为了防止数字过小溢出
    # p0num = np.zeros(words_num)
    # p1num = np.zeros(words_num)
    p0num = np.ones(words_num)
    p1num = np.ones(words_num)
    # 整个数据集单词出现的次数（原来是0，后面改成2了）
    p0num_all = 2.0
    p1num_all = 2.0

    for i in range(train_doc_num):
        # 遍历所有的文件，如果是侮辱性文件，就计算此侮辱性文件中出现的侮辱性单词的个数
        if train_category[i] == 1:
            p1num += train_mat[i]
            p1num_all += np.sum(train_mat[i])
        else:
            p0num += train_mat[i]
            p0num_all += np.sum(train_mat[i])
    # 后面改成取 log 函数
    p1vec = np.log(p1num / p1num_all)
    p0vec = np.log(p0num / p0num_all)
    return p0vec, p1vec, pos_abusive


def classify_naive_bayes(vec2classify, p0vec, p1vec, p_class1):
    """
    使用算法：
        # 将乘法转换为加法
        乘法：P(C|F1F2...Fn) = P(F1F2...Fn|C)P(C)/P(F1F2...Fn)
        加法：P(F1|C)*P(F2|C)....P(Fn|C)P(C) -> log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))
    :param vec2classify: 待测数据[0,1,1,1,1...]，即要分类的向量
    :param p0vec: 类别0，即正常文档的[log(P(F1|C0)),log(P(F2|C0)),log(P(F3|C0)),log(P(F4|C0)),log(P(F5|C0))....]列表
    :param p1vec: 类别1，即侮辱性文档的[log(P(F1|C1)),log(P(F2|C1)),log(P(F3|C1)),log(P(F4|C1)),log(P(F5|C1))....]列表
    :param p_class1: 类别1，侮辱性文件的出现概率
    :return: 类别1 or 0
    """
    # 计算公式  log(P(F1|C))+log(P(F2|C))+....+log(P(Fn|C))+log(P(C))
    # 使用 NumPy 数组来计算两个向量相乘的结果，这里的相乘是指对应元素相乘，即先将两个向量中的第一个元素相乘，然后将第2个元素相乘，以此类推。
    # 我的理解是：这里的 vec2Classify * p1Vec 的意思就是将每个词与其对应的概率相关联起来
    # 可以理解为 1.单词在词汇表中的条件下，文件是good 类别的概率 也可以理解为 2.在整个空间下，文件既在词汇表中又是good类别的概率
    p1 = np.sum(vec2classify * p1vec) + np.log(p_class1)
    p0 = np.sum(vec2classify * p0vec) + np.log(1 - p_class1)
    if p1 > p0:
        return 1
    else:
        return 0

def create_vocab_list(data_set):
    """
    获取所有单词的集合
    :param data_set: 数据集
    :return: 所有单词的集合(即不含重复元素的单词列表)
    """
    vocab_set = set()  # create empty set
    for item in data_set:
        # | 求两个集合的并集
        vocab_set = vocab_set | set(item)
    return list(vocab_set)
# 接收一个大写字符串并将其解析成字符串列表
def text_parse(big_str):
    """
    这里就是做词划分
    :param big_str: 某个被拼接后的字符串
    :return: 全部是小写的word列表，去掉少于 2 个字符的字符串
    """
    import re
    # 其实这里比较推荐用　\W+ 代替 \W*，
    # 因为 \W*会match empty patten，在py3.5+之后就会出现什么问题，推荐自己修改尝试一下，可能就会re.split理解更深了
    token_list = re.split(r'\W+', big_str)
    if len(token_list) == 0:
        print(token_list)
    return [tok.lower() for tok in token_list if len(tok) > 2]

# 导入文件夹中数据，并将它们解析成词列表
def spam_test():
    """
    对贝叶斯垃圾邮件分类器进行自动化处理。
    :return: nothing
    """
    doc_list = []
    class_list = []# 标识列表，这里是标识侮辱和非侮辱的词语
    full_text = []
    #添加文档
    doc_list,full_text,class_list = openFile(r'C:\Users\admin\Desktop\machine-learning\ai\datasets\YoutubeCommentsSpam\YoutubeCommentsSpam_train.csv')
    # 创建词汇表
    # print(1)
    vocab_list = create_vocab_list(doc_list)#创建一个词汇表

    # import random
    # 生成随机取10个数, 为了避免警告将每个数都转换为整型
    # test_set = [int(num) for num in random.sample(range(1853), 300)]
    # 并在原来的training_set中去掉这10个数
    print(len(class_list))
    training_set = list(set(range(len(class_list)-1)))
    training_mat = []
    training_class = []
    print(len(doc_list))
    for doc_index in training_set:
        # print(doc_index)
        training_mat.append(set_of_words2vec(vocab_list, doc_list[doc_index]))#转化为词向量
        training_class.append(class_list[doc_index])# 将0,1等提出来
    p0v, p1v, p_spam = train_naive_bayes(
        np.array(training_mat),
        np.array(training_class)
    )#通过训练集的到预测函数

    error_count = 0
    i = 1
    doc_list1 = []
    full_text1 = []
    class_list1 = []
    doc_list1,full_text1,class_list1 = openFile(r'C:\Users\admin\Desktop\machine-learning\ai\datasets\YoutubeCommentsSpam\YoutubeCommentsSpam_test.csv')
    print(len(class_list1))
    testset = list(range(len(class_list1)-1))
    for doc_index in testset:

        word_vec = set_of_words2vec(vocab_list, doc_list1[doc_index])
        if classify_naive_bayes(
                np.array(word_vec),
                p0v,
                p1v,
                p_spam
        ) != class_list1[doc_index]:#这个就是预测结果
            print(doc_list1[doc_index])
            error_count += 1
        # print(word_vec)
    print('the error rate is {}'.format(
        error_count / len(testset)
    ))
def openFile(round:str):
    doc_list = []
    class_list = []# 标识列表，这里是标识侮辱和非侮辱的词语
    full_text = []
    csv_file = open(round)
    csv_reader_lines = csv.reader(csv_file)  # 逐行读取csv文件
    print(type(csv_reader_lines))
    date = []  # 创建列表准备接收csv各行数据
    for one_line in csv_reader_lines:
        if(re.search(u"[\u4e00-\u9fa5]+",one_line[0])==None):#将中文乱码抹去
            date.append(one_line)  # 将读取的csv分行数据按行存入列表‘date’中
    i = 1
    while i < len(date):
        words = text_parse(date[i][0])
        doc_list.append(words)
        full_text.extend(words)
        # print(date[i][1])
        class_list.append(int(date[i][1]))
        i+=1
    return  doc_list,full_text,class_list
spam_test()

<class '_csv.reader'>
1713
1713
<class '_csv.reader'>
105
['how', 'did', 'you', 'know', 'that', 'people', 'makes', 'another', 'account', 'just', 'for', 'subscribing', 'itself', 'and', 'liking']
['gonna', 'share', 'little', 'ryhme', 'canibus', 'blows', 'eminem', 'away', 'quadrillion', 'times', 'especially', 'about', 'the', 'categories', 'intelligent', 'things', 'his', 'mind', 'that', 'learned', 'and', 'rapped', 'about', 'and', 'forgot', 'before', 'eminem', 'spit', 'his', 'first', 'ryme', 'luv', 'linz']
['753', '682', '421', 'gangnam', 'style']
['some', 'classsic']
['hey', 'guys', 'love', 'this', 'but', 'check', 'out', 'this', 'girl', 'name', 'cause', 'she', 'knows', 'how', 'dance', 'search', 'dancing']
['part', 'holy', 'mary', 'pray', 'for', 'holy', 'mother', 'god', 'pray', 'for', 'holy', 'virgin', 'virgins', 'pray', 'for', 'mother', 'christ', 'pray', 'for', 'mother', 'divine', 'grace', 'pray', 'for', 'mother', 'most', 'pure', 'pray', 'for', 'mother', 'most', 'chaste', 'pray', 'for', 'm